In [15]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017")
db = client.sample_mflix   # = client["sample_mflix"] , 온점표기법에서 .이나 숫자로 시작하면 사용X => 대괄호표기법 사용

users = db.users
comments = db.comments
movies = db.movies

print("OK", db.name, "collections:", [c for c in db.list_collection_names() if c in{"users", "comments", "movies"}])  
# 반복문 결과값을 출력하기 위해 for 앞에 c 넣음  => c는 전체 컬렉션
# {} set : 값이 중복되지 않은 자료 형태  => set이 매칭되어져있다면

OK sample_mflix collections: ['users', 'movies', 'comments']


In [21]:
# 유저&커멘트 연결 -> 커멘트랑 조인 -> 커멘트 text 길이

loyal_users_PipeLine = [
    {
        "$lookup": {
            "from" : "comments",
            "localField" : "email",
            "foreignField" : "email",
            "as" : "c"
        }
    },
    {
        "$addFields" : {   # 새필드를 사용자 문서에 추가
            "commentsCount" : {"$size": "$c"},  # size : 배열의 길이를 계산하는 연산자 
            "avgTextLen" : {
                "$avg" : {
                    "$map": {  # 배열을 변환
                        "input" : "$c",  # 댓글 배열을 순회
                        "as" : "x",  # 각 댓글 문서를 x라 부름
                        "in" : {
                            "$strLenCP" : {   # 문자열 길이 계산 
                                "$ifNull": ["$$x.text", ""]   # null 방지 : 댓글 text가 없으면 빈 문자열로 처리 
                            }
                        }
                    }
                }
            },
            "lastCommentDate": { "$max": "$c.date" }  
        }
    },
    {"$match" : {"commentsCount" : {"$gte" : 10}}},
    {"$sort" : {"commentsCount" : -1, "avgTextLen" : -1, "lastCommentDate" : -1}},
    {
        "$project" : {
            "_id" : 0, 
            "name" : 1, 
            "email" : 1, 
            "commentsCount" : 1, 
            "avgTextLen" : 1, 
            "lastCommentDate" : 1
        }
    }
]

loyal_users = list(users.aggregate(loyal_users_PipeLine))
len(loyal_users), loyal_users[:5]

# NoSQL => 현업 실무 개발자들은 JOIN -> 서로 다른 데이터 테이블 혹은 컬렉션을 하나로 연결해서 가져와 처리하면 해당 테이터의 양이 방대해져
# 방대해진 데이터를 처리하기 위한 하드웨어에 많은 무리가 가고 기계 성능 & 사양 제약

(183,
 [{'name': 'Mace Tyrell',
   'email': 'roger_ashton-griffiths@gameofthron.es',
   'commentsCount': 331,
   'avgTextLen': 152.49244712990938,
   'lastCommentDate': datetime.datetime(2017, 8, 2, 1, 37, 26)},
  {'name': 'Missandei',
   'email': 'nathalie_emmanuel@gameofthron.es',
   'commentsCount': 327,
   'avgTextLen': 153.0948012232416,
   'lastCommentDate': datetime.datetime(2017, 9, 11, 16, 52, 51)},
  {'name': 'The High Sparrow',
   'email': 'jonathan_pryce@gameofthron.es',
   'commentsCount': 315,
   'avgTextLen': 152.68253968253967,
   'lastCommentDate': datetime.datetime(2017, 7, 10, 4, 58, 23)},
  {'name': 'Sansa Stark',
   'email': 'sophie_turner@gameofthron.es',
   'commentsCount': 308,
   'avgTextLen': 153.42532467532467,
   'lastCommentDate': datetime.datetime(2017, 5, 2, 16, 29, 50)},
  {'name': 'Rodrik Cassel',
   'email': 'ron_donachie@gameofthron.es',
   'commentsCount': 305,
   'avgTextLen': 151.75737704918032,
   'lastCommentDate': datetime.datetime(2017, 9, 2, 6

In [24]:
movies_report_PipeLine = [
    {
        "$facet" : {
            "latest5" : [
                {"$sort" : {"year" : -1}},
                {"$limit" : 5},
                {"$project" : {"_id" : 0, "title" : 1, "year" : 1}}
            ],
            "highRatedCount" : [
                {"$match" : {"imdb.rating" : {"$gte" : 8}}},  # imdb.rating : 비교
                {"$count" : "count"}
            ],
            "genresTop10" : [
                {"$unwind" : "$genres"},
                {"$group" : {"_id" : "$genres", "count" : {"$sum" : 1}}},
                {"$sort" : {"count" : -1}},
                {"$limit" : 10}
            ],
            "yearlyAvgRecent10" : [
                {"$group" : {"_id" : "$year", "avgRating" : {"$avg" : "$imdb.rating"}}},  # $imdb.rating : 사칙연산하기위함
                {"$sort" : {"_id" : -1}},
                {"$limit" : 10},
                {"$sort" : {"_id" : -1}},
                {"$project" : {
                    "_id" : 0, "year" : "$_id", "avgRating" : 1, 
                }}
            ]
        }
    }
]
movie_report = list(movies.aggregate(movies_report_PipeLine))
movie_report

[{'latest5': [{'title': 'The Saboteurs', 'year': '2015è'},
   {'title': 'Halo: Nightfall', 'year': '2014è'},
   {'title': 'The Roosevelts: An Intimate History', 'year': '2014è'},
   {'title': 'The Weight of the Nation', 'year': '2012è'},
   {'title': 'Hit & Miss', 'year': '2012è'}],
  'highRatedCount': [{'count': 1596}],
  'genresTop10': [{'_id': 'Drama', 'count': 13789},
   {'_id': 'Comedy', 'count': 7025},
   {'_id': 'Romance', 'count': 3665},
   {'_id': 'Crime', 'count': 2678},
   {'_id': 'Thriller', 'count': 2658},
   {'_id': 'Action', 'count': 2540},
   {'_id': 'Documentary', 'count': 2129},
   {'_id': 'Adventure', 'count': 2045},
   {'_id': 'Horror', 'count': 1703},
   {'_id': 'Biography', 'count': 1404}],
  'yearlyAvgRecent10': [{'avgRating': 8.4, 'year': '2015è'},
   {'avgRating': 7.25, 'year': '2014è'},
   {'avgRating': 7.5, 'year': '2012è'},
   {'avgRating': 7.75, 'year': '2011è'},
   {'avgRating': 8.2, 'year': '2010è'},
   {'avgRating': 8.3, 'year': '2009è'},
   {'avgRating'